In [1]:

import sys
from pathlib import Path

project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import src.seed as seed
import src.models as models
import src.functions as fn

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
device = seed.device
generator = seed.generator

X, y, X_test, y_test = fn.load_cifar_10()

100%|██████████| 170M/170M [00:09<00:00, 17.8MB/s] 


Extracting C:\Users\Eric\DSC Capstone\Investigating-NN-Optimizers\data\cifar-10-python.tar.gz to C:\Users\Eric\DSC Capstone\Investigating-NN-Optimizers\data
Files already downloaded and verified


C:\Users\Eric\DSC Capstone\Investigating-NN-Optimizers\src\functions.py:80: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:28.)
  X = torch.tensor(X, dtype=torch.float32, device=device)


In [7]:
input_size = X.shape[1] * X.shape[2] * X.shape[3]
num_hidden_layers = 2
hidden_layer_size = 200

output_dir = "eos/sgd_EC"
num_labels = 10

In [8]:
epochs = 20000
learning_rate = 0.05
accuracy = 0.99

model = models.FullyConnectedNet(
    input_size=input_size,
    num_hidden_layers=num_hidden_layers,
    hidden_layer_size=hidden_layer_size,
    num_labels=10,
    activation=nn.Tanh
)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.0)

fn.train_model(model, optimizer, criterion, epochs, accuracy, X, y, X_test, y_test, output_dir)

Training FullyConnectedNet with SGD and learning rate 0.05 for 20000 epochs.
Epoch [1000/20000], Loss: 0.0639, Time: 0.04, Train Acc: 0.6652, Test Acc: 0.2810, 
Epoch [2000/20000], Loss: 0.0509, Time: 0.07, Train Acc: 0.8162, Test Acc: 0.3090, 
Epoch [3000/20000], Loss: 0.0424, Time: 0.1, Train Acc: 0.8864, Test Acc: 0.3170, 
Epoch [4000/20000], Loss: 0.0357, Time: 0.13, Train Acc: 0.9254, Test Acc: 0.3150, 
Epoch [5000/20000], Loss: 0.0303, Time: 0.16, Train Acc: 0.9494, Test Acc: 0.3140, 
Epoch [6000/20000], Loss: 0.0258, Time: 0.2, Train Acc: 0.9676, Test Acc: 0.3090, 
Epoch [7000/20000], Loss: 0.0221, Time: 0.23, Train Acc: 0.9784, Test Acc: 0.2930, 
Epoch [8000/20000], Loss: 0.0194, Time: 0.26, Train Acc: 0.9842, Test Acc: 0.2840, 
Epoch [9000/20000], Loss: 0.0169, Time: 0.3, Train Acc: 0.9894, Test Acc: 0.2790, 


In [9]:
md, out = fn.load_output_files(output_dir)

In [10]:
out

,model_id,epoch,train_loss,train_accuracy,test_accuracy,sharpness_H,sharpness_A
0,1.0,1,0.568510,0.1026,0.108,78.3402,NaN
1,1.0,2,0.492131,0.1086,0.121,NaN,NaN
2,1.0,3,0.696503,0.1124,0.093,NaN,NaN
3,1.0,4,0.944314,0.1096,0.127,NaN,NaN
4,1.0,5,0.775662,0.1134,0.094,NaN,NaN
...,...,...,...,...,...,...,...
19995,1.0,19996,NaN,NaN,NaN,NaN,NaN
19996,1.0,19997,NaN,NaN,NaN,NaN,NaN
19997,1.0,19998,NaN,NaN,NaN,NaN,NaN
19998,1.0,19999,NaN,NaN,NaN,NaN,NaN


In [12]:
import plotly.express as px

fig = px.scatter(out, x='epoch', y='sharpness_H')
fig.add_hline(y=2/learning_rate, line_dash="dash", line_color="red")
fig.show()

In [14]:
fig = px.line(
    out,
    x="epoch",
    y=["train_accuracy", "test_accuracy"],
    markers=True,
)

fig.show()